Caso de Estudio FitWell App

actividad_usuarios.csv → Eventos en la app (estructurado)
suscripciones_usuarios.xlsx → Estado y pagos (estructurado)
progreso_planes.csv → Progreso en los planes (estructurado)
interacciones_nutricion.xlsx → Métricas de nutrición (estructurado)
datos_demograficos.json → Perfil de usuario (no estructurado)

In [3]:
import pandas as pd           # Para manipular datos
import numpy as np            # Para operaciones matemáticas
import matplotlib.pyplot as plt  # Para gráficos
import seaborn as sns         # Para gráficos más bonitos



In [8]:

# Definimos la ruta a tus archivos originales
path_raw = r'C:\Users\ari90\Desktop\TUP-UTN\Materias\Intro-Analisis-Datos\Proyecto-FitnessApp\data\raw'

# 1. Carga de datos originales
actividad_raw = pd.read_csv(f'{path_raw}/actividad_usuarios.csv')
demografia_raw = pd.read_json(f'{path_raw}/datos_demograficos.json')
suscripciones_raw = pd.read_excel(f'{path_raw}/suscripciones_usuarios.xlsx')
progreso_raw = pd.read_csv(f'{path_raw}/progreso_planes.csv')
nutricion_raw = pd.read_excel(f'{path_raw}/interacciones_nutricion.xlsx')

In [9]:
# 2. Crear copias de seguridad en memoria (Usa .copy() para que sean objetos independientes)
df_actividad = actividad_raw.copy()
df_demografia = demografia_raw.copy()
df_suscripciones = suscripciones_raw.copy()
df_progreso = progreso_raw.copy()
df_nutricion = nutricion_raw.copy()

print("✅ Copias de trabajo creadas. Los archivos originales en 'raw' están protegidos.")

✅ Copias de trabajo creadas. Los archivos originales en 'raw' están protegidos.
